In [1]:
from pyspark.sql import SparkSession

In [2]:
speark = SparkSession.builder.appName('linearReg').getOrCreate()

In [3]:
df = spark.read.csv('/FileStore/tables/cruise_ship_info.csv', header=True, inferSchema=True)

In [4]:
df.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [5]:
df.head(5)

Out[ 5 ]: 
[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1),
 Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)]

In [6]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
 Cruise_line|count|
+-----------------+-----+
 Costa| 11|
 P&O| 6|
 Cunard| 3|
Regent_Seven_Seas| 5|
 MSC| 8|
 Carnival| 22|
 Crystal| 2|
 Orient| 1|
 Princess| 17|
 Silversea| 4|
 Seabourn| 3|
 Holland_American| 14|
 Windstar| 3|
 Disney| 2|
 Norwegian| 13|
 Oceania| 3|
 Azamara| 2|
 Celebrity| 10|
 Star| 6|
 Royal_Caribbean| 23|
+-----------------+-----+

In [7]:
from pyspark.ml.feature import StringIndexer

In [8]:
strIndexer = StringIndexer(inputCol='Cruise_line', outputCol='cruise_cat')
indexed = strIndexer.fit(df).transform(df)

In [9]:
indexed.head(1)

Out[ 9 ]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)]

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
indexed.columns

Out[ 11 ]: 
['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [12]:
assembler = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density', 'crew', 'cruise_cat'], outputCol='features')

In [13]:
output = assembler.transform(indexed)

In [14]:
output.select(['features', 'crew']).show()

+--------------------+----+
 features|crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [15]:
final_df = output.select(['features', 'crew'])

In [16]:
train_data, test_data = final_df.randomSplit([0.7,0.3])

In [17]:
train_data.describe().show()

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 109|
 mean|7.750275229357809|
 stddev|3.316153913104644|
 min| 0.88|
 max| 21.0|
+-------+-----------------+

In [18]:
test_data.describe().show()

+-------+------------------+
summary| crew|
+-------+------------------+
 count| 49|
 mean| 7.891836734693876|
 stddev|3.9230715397660605|
 min| 0.59|
 max| 19.1|
+-------+------------------+

In [19]:
from pyspark.ml.regression import LinearRegression

In [20]:
lr = LinearRegression(labelCol='crew')

In [21]:
model = lr.fit(train_data)

In [22]:
results = model.evaluate(test_data)

In [23]:
results.rootMeanSquaredError

Out[ 23 ]: 0.0

In [24]:
results.r2

Out[ 24 ]: 1.0

In [25]:
results.meanSquaredError

Out[ 25 ]: 0.0

In [26]:
from pyspark.sql.functions import corr

In [27]:
df.select(corr('crew','cabins')).show()

+------------------+
corr(crew, cabins)|
+------------------+
0.9508226063578497|
+------------------+